### IMPORT

In [1]:
import cv2
import numpy as np
import os

### HELPER FUCTION

In [2]:
# Fungsi untuk mengubah text menjadi bit
def text_to_bits(text):
    return ''.join(format(ord(c), '08b') for c in text)

# Fungsi untuk mengubah bit menjadi text
def bits_to_text(bits):
    chars = []
    for i in range(0, len(bits), 8):
        byte = bits[i:i+8]
        chars.append(chr(int(byte, 2)))
    return ''.join(chars)

### Convert JPG to PNG 

In [3]:
def convert_jpg_to_png(folder_in, folder_out):
    os.makedirs(folder_out, exist_ok=True)

    for filename in os.listdir(folder_in):
        if filename.lower().endswith((".jpg", ".jpeg")):
            path = os.path.join(folder_in, filename)
            img = cv2.imread(path)
            if img is None:
                print("gagal membaca:", filename)
                continue
            name = os.path.splitext(filename)[0]
            out_path = os.path.join(folder_out, name + ".png")
            cv2.imwrite(out_path, img)
            print("converted", out_path)

        elif filename.lower().endswith(".png"):
            src = os.path.join(folder_in, filename)
            dst = os.path.join(folder_out, filename)
            cv2.imwrite(dst, cv2.imread(src))
            print("copied", dst)

    print("Konversi JPG → PNG selesai\n")

### LSB Embed ke Citra Batik

In [4]:
def embed_lsb(image_path, message, output_path="stego.png"):

    # Load image
    img = cv2.imread(image_path)
    if img is None:
        print(f"gagal membaca gambar: {image_path}")
        return None

    height, width, channel = img.shape

    # Ubah pesan ke bit
    bits = text_to_bits(message)
    msg_len = len(bits)

    # 32-bit header = panjang pesan
    header = format(msg_len, '032b')
    payload = header + bits

    # Flatten image
    flat = img.flatten()
    capacity = len(flat)

    # Cek kapasitas gambar terhadap panjang pesan
    if len(payload) > capacity:
        print("pesan terlalu panjang untuk gambar ini!")
        return None
    
    # Sisipkan payload ke flatten image
    for i in range(len(payload)):
        flat[i] = (flat[i] & 0xFE) | int(payload[i])

    stego = flat.reshape((height, width, channel))
    cv2.imwrite(output_path, stego)
    print("pesan berhasil disisipkan", output_path)

    return stego

### Esktrak Hasil LSB dari Citra

In [5]:
def extract_lsb(image_path):
    # Load image
    img = cv2.imread(image_path)
    if img is None:
        print(f"gagal membaca gambar: {image_path}")
        return None
    
    # Flatten image
    flat = img.flatten()
    total = len(flat)

    # Baca header (32 bit)
    header_bits = ""
    for i in range(32):
        header_bits += str(flat[i] & 1)

    msg_len = int(header_bits, 2)

    # msg_len <= 0, maka tidak valid
    if msg_len <= 0:
        return "(Invalid payload)"

    # kapasitas minimal, yaitu 32 header + msg_len
    if 32 + msg_len > total:
        return "(Invalid payload)"

    bits = ""
    end = 32 + msg_len

    for i in range(32, end):
        bits += str(flat[i] & 1)

    # Jika bits kurang, maka corrupted
    if len(bits) != msg_len:
        return "(Corrupted payload)"

    return bits_to_text(bits)


### Fungsi Menghitung Peak Signal-to-Noise Ratio

In [6]:
def psnr(img1, img2):
    mse = np.mean((img1 - img2) ** 2)
    if mse == 0:
        return 999
    PIXEL_MAX = 255.0
    return 20 * np.log10(PIXEL_MAX / np.sqrt(mse))

### Batch Embedding untuk image

In [7]:
def batch_embed(folder_in, folder_out, message):
    os.makedirs(folder_out, exist_ok=True)

    for filename in os.listdir(folder_in):
        if filename.lower().endswith(".png"):
            inp = os.path.join(folder_in, filename)
            name = os.path.splitext(filename)[0]
            outp = os.path.join(folder_out, f"stego_{name}.png")
            embed_lsb(inp, message, outp)

    print("batch embed selesai\n")

### Batch Extract untuk image

In [8]:
def batch_extract(folder_original, folder_stego):
    results = []

    for filename in os.listdir(folder_stego):
        if filename.lower().endswith(".png"):

            stego_path = os.path.join(folder_stego, filename)
            base = filename.replace("stego_", "").replace(".png","")
            
            ori_path = None
            for ext in [".png", ".jpg", ".jpeg"]:
                test = os.path.join(folder_original, base + ext)
                if os.path.exists(test):
                    ori_path = test
                    break

            msg = extract_lsb(stego_path)

            ori = cv2.imread(ori_path) if ori_path else None
            ste = cv2.imread(stego_path)

            psnr_val = psnr(ori, ste) if ori is not None else None

            results.append((filename, msg, psnr_val))

    return results

### Main Fuction

In [9]:
if __name__ == "__main__":

    convert_jpg_to_png("dataset_batik", "dataset_png")

    batch_embed(
        "dataset_png",
        "stego_batik",
        "Batik merupakan salah satu warisan budaya Indonesia yang memiliki nilai sejarah dan filosofi yang sangat tinggi. "
        "Setiap motif batik tidak hanya menghadirkan keindahan visual, tetapi juga membawa pesan dan makna yang mendalam. "
        "Motif parang, misalnya, menggambarkan kekuatan, semangat pantang menyerah, serta keberanian dalam menghadapi tantangan hidup. "
        "Sementara itu, motif kawung dipercaya melambangkan kesucian hati dan keinginan manusia untuk mencapai keseimbangan hidup."
    )

    res = batch_extract("dataset_png", "stego_batik")

    print("\nHasil batch extract + PSNR:")
    for fname, msg, val_psnr in res:

        # --- klasifikasi PSNR ---
        if val_psnr is None:
            kualitas = "Tidak dapat dihitung"
        elif val_psnr > 50:
            kualitas = "Sangat sangat baik (perubahan hampir mustahil terlihat)"
        elif 40 <= val_psnr <= 50:
            kualitas = "Sangat baik (perubahan tidak terlihat mata)"
        elif 30 <= val_psnr < 40:
            kualitas = "Baik, perubahan sangat kecil"
        elif 25 <= val_psnr < 30:
            kualitas = "Terlihat sedikit perubahan"
        else:
            kualitas = "Kualitas buruk, perubahan terlihat jelas"

        print(f"{fname:<25} → PSNR: {val_psnr:<8} | Kualitas: {kualitas}")
        # print(f"Pesan: {msg}\n") # Kalau ingin lihat pesan


converted dataset_png\1.png
converted dataset_png\10.png
converted dataset_png\11.png
converted dataset_png\12.png
converted dataset_png\13.png
converted dataset_png\14.png
converted dataset_png\15.png
converted dataset_png\16.png
converted dataset_png\17.png
converted dataset_png\18.png
converted dataset_png\19.png
converted dataset_png\2.png
converted dataset_png\20.png
converted dataset_png\21.png
converted dataset_png\22.png
converted dataset_png\23.png
converted dataset_png\24.png
converted dataset_png\25.png
converted dataset_png\26.png
converted dataset_png\27.png
converted dataset_png\28.png
converted dataset_png\29.png
converted dataset_png\3.png
converted dataset_png\30.png
converted dataset_png\31.png
converted dataset_png\32.png
converted dataset_png\33.png
converted dataset_png\34.png
converted dataset_png\35.png
converted dataset_png\36.png
converted dataset_png\37.png
converted dataset_png\38.png
converted dataset_png\39.png
converted dataset_png\4.png
converted dataset_